In [561]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import os
from shapely.geometry import MultiPoint
from shapely.geometry import Polygon
import sys
import plotly.express as px
import geopandas as gpd
import plotly
import plotly.graph_objects as go
from scipy.spatial import distance
import statsmodels.api as sm
import xgboost as xgb
from urllib.request import urlopen
import json
import pickle
from joblib import dump, load

In [45]:
# load data
RANDOM_STATE = 123
fire_df = pd.read_csv(r"Fire_Incidents.csv")
green_light = pd.read_csv(r"Project_Green_Light_Locations.csv")
green_light['live_date'] = pd.to_datetime(green_light['live_date'])
dfd_locations = pd.read_csv(r"DFD_Fire_Station_Locations.csv")

In [3]:
fire_df.head()

,exposure,incident_address,incident_number,incident_type_desc,property_use,engine_area,call_datetime,dispatch_datetime,arrival_datetime,cleared_datetime,civilian_injury,civilian_fatality,fire_injury,fire_fatality,structure_status,x,y,oid,geom
0,No,2409 TOWNSEND ST,16-0030216,Building fire ...,NaN,E09,2016/09/29 11:46:41+00,2016/09/29 11:49:17+00,2016/09/29 11:55:11+00,2016/09/29 12:54:39+00,0,0,0,0,NaN,-83.005263,42.359048,1,NaN
1,No,7874 FRONTENAC ST,16-0030244,Flood assessment ...,NaN,E09,2016/09/29 13:59:31+00,2016/09/29 14:06:17+00,2016/09/29 14:18:54+00,2016/09/29 14:26:22+00,0,0,0,0,NaN,-83.028054,42.387677,2,NaN
2,No,1881 E GRAND BLVD,16-0030266,Gas leak (natural gas or LPG) ...,NaN,E09,2016/09/29 16:19:25+00,2016/09/29 16:21:32+00,2016/09/29 16:30:24+00,2016/09/29 16:35:53+00,0,0,0,0,NaN,-83.038307,42.374396,3,NaN
3,No,1265 MELDRUM ST,16-0030301,Swift water rescue ...,NaN,E09,2016/09/29 20:57:20+00,2016/09/29 21:00:09+00,2016/09/29 21:04:30+00,2016/09/29 21:17:44+00,0,0,0,0,NaN,-83.011810,42.348770,4,NaN
4,No,17154 LAMONT ST,16-0031935,Building fire ...,NaN,E44,2016/10/12 00:00:14+00,2016/10/12 00:03:20+00,2016/10/12 00:07:08+00,2016/10/12 02:21:52+00,0,0,0,0,NaN,-83.053816,42.419416,5,NaN


In [38]:
# clean/transform data
# long = x, lat = y
fire_inc = fire_df.copy()
fire_inc['call_datetime'] = pd.to_datetime(fire_inc['call_datetime'])
fire_inc['injury_or_fatality'] = fire_inc.apply(
                                    lambda x: 1 if (x['civilian_injury'] > 0) | 
                                    (x['civilian_fatality'] > 0) | (x['fire_injury'] > 0) |
                                    (x['fire_fatality'] > 0) else 0, axis=1)

fire_inc.dropna(subset=['x', 'y'], inplace=True)

cols = ['injury_or_fatality','call_datetime', 'x', 'y']
fire_inc = fire_inc[cols]

In [ ]:
#update feature columns
fire_inc['hour'] = fire_inc['call_datetime'].dt.hour
fire_inc['day'] = fire_inc['call_datetime'].dt.day
fire_inc['DoY'] = fire_inc['call_datetime'].dt.dayofyear
fire_inc['DoW'] = fire_inc['call_datetime'].dt.dayofweek
fire_inc['week'] = fire_inc['call_datetime'].dt.isocalendar().week
fire_inc['month'] = fire_inc['call_datetime'].dt.month
fire_inc['year'] = fire_inc['call_datetime'].dt.year
fire_inc['yearMonth'] = fire_inc['year']*100 + fire_inc['month']
fire_inc['weekend'] = fire_inc['DoW'].apply(lambda x: 1 if (x == 0) | (x == 6) else 0)

#filter to full years
START = 2017
END = 2019
fire_inc = fire_inc[(fire_inc['year'] >= START) & (fire_inc['year'] <= END)]

#use sin/cos functions to capture periodicity
numPeriods = {'hour': 24, 'day': 31, 'DoY': 365, 'DoW': 7, 'week': 53, 'month': 12}
for colName in ['hour','day','DoY','DoW','week','month']:
    fire_inc[colName + 'x'] = np.sin(2 * np.pi * fire_inc[colName] / numPeriods[colName])
    fire_inc[colName + 'y'] = np.cos(2 * np.pi * fire_inc[colName] / numPeriods[colName])

fatal = fire_inc[fire_inc['injury_or_fatality'] == 1]
dfs = {'fatal': fatal, 'all': fire_inc}

In [564]:
datetime.date('2018-02-02')

TypeError: descriptor 'date' for 'datetime.datetime' objects doesn't apply to a 'str' object

In [40]:
##add closest fire station
dfd_coords = np.array(list(zip(dfd_locations['X'], dfd_locations['Y'])))
fire_inc['closest_stn'] = fire_inc.apply(lambda x: distance.cdist(np.array([(x['x'], x['y'])]), dfd_coords).min(axis=1)[0], axis=1)

In [46]:
green_light.head()

,X,Y,address,business_name,business_type,precinct,live_date,ObjectId
0,-83.199179,42.401723,15510 Fenkell St,Mobil,Retail,8,2016-01-01 05:00:00+00:00,1
1,-83.159842,42.417175,8930 W McNichols Rd,Marathon,Retail,12,2016-01-01 05:00:00+00:00,2
2,-83.197639,42.386789,15439 Schoolcraft Ave,Citgo,Retail,2,2016-01-01 05:00:00+00:00,3
3,-83.004221,42.434677,11603 E 7 Mile Rd,Sunoco,Retail,9,2016-01-01 05:00:00+00:00,4
4,-82.959684,42.405842,10601 E Outer Dr,BP,Retail,9,2016-01-01 05:00:00+00:00,5


In [47]:
##use function to find nearest greenlight - we nned to filter lights to those that were live before incident
def find_closest_green_light(inc_df, green_light_df):
    live_lights = green_light_df[green_light_df['live_date'] < inc_df['call_datetime']]
    light_coords = np.array(list(zip(live_lights['X'], live_lights['Y'])))
    inc_coords = np.array([(inc_df['x'], inc_df['y'])])
    min_dist = distance.cdist(inc_coords, light_coords).min(axis=1)[0]
    return min_dist
fire_inc['closest_light'] = fire_inc.apply(lambda x: find_closest_green_light(x, green_light), axis=1)

In [49]:
fire_inc.head()

,injury_or_fatality,call_datetime,x,y,hour,day,DoY,DoW,week,month,...,DoYx,DoYy,DoWx,DoWy,weekx,weeky,monthx,monthy,closest_stn,closest_light
6,0,2017-01-04 00:10:07+00:00,-83.088720,42.426064,0,4,4,2,1,1,...,0.068802,0.997630,0.974928,-0.222521,0.118273,0.992981,0.5,8.660254e-01,0.015584,0.015269
7,1,2017-01-10 09:23:12+00:00,-83.082236,42.418395,9,10,10,1,2,1,...,0.171293,0.985220,0.781831,0.623490,0.234886,0.972023,0.5,8.660254e-01,0.022253,0.016398
8,0,2017-01-14 08:03:22+00:00,-83.099447,42.443401,8,14,14,5,2,1,...,0.238673,0.971100,-0.974928,-0.222521,0.234886,0.972023,0.5,8.660254e-01,0.011979,0.020578
9,0,2017-03-02 16:32:33+00:00,-83.135443,42.380273,16,2,61,3,9,3,...,0.867456,0.497513,0.433884,-0.900969,0.875735,0.482792,1.0,6.123234e-17,0.013263,0.011673
10,0,2017-03-06 20:32:38+00:00,-83.045696,42.407905,20,6,65,0,10,3,...,0.899631,0.436651,0.000000,1.000000,0.926690,0.375828,1.0,6.123234e-17,0.016332,0.035715


In [530]:
#Kmeans clustering on incidents
CLUSTERS = 20 # num of fire stations in city
clustering = KMeans(CLUSTERS)
clustering.fit(fire_inc[['y', 'x']])
fire_inc['cluster'] = clustering.labels_

#one-hot encode cluster
#fire_inc = pd.get_dummies(data=fire_inc, columns=['cluster'], prefix='cluster')



In [551]:
#Kmeans clustering on injuries
CLUSTERS = 20
clustering = KMeans(CLUSTERS)
clustering.fit(fire_inc[fire_inc['injury_or_fatality'] == 1][['y', 'x']])
fire_inc['cluster'] = clustering.predict(fire_inc[['y', 'x']])

In [371]:
clustering = DBSCAN(eps=.0015, min_samples=6).fit(fire_inc[['y', 'x']])
fire_inc['cluster'] = clustering.labels_


In [562]:
dump(clustering, "cluster_model.joblib")

['cluster_model.joblib']

In [580]:
clustering.predict([[42.4347, -83.0042]])[0]   # long = x, lat = y

17

In [552]:
#create polygon around each cluster so we an visualize it
clusters = {'cluster': [], 'geometry': [], 'incident_count': [], 'inj/inc': [], 'inj': []}
for cluster in fire_inc['cluster'].unique():
    if cluster == -1:
        continue
    points = fire_inc[fire_inc['cluster'] == cluster]
    clusterPoints = MultiPoint(list(zip(points['x'], points['y'])))
    clusterPoly = clusterPoints.convex_hull
    clusters['cluster'].append(cluster)
    clusters['geometry'].append(clusterPoly)
    clusters['incident_count'].append(len(points))
    clusters['inj/inc'].append(
                            len(points[points['injury_or_fatality'] == 1])/len(points))
    clusters['inj'].append(len(points[points['injury_or_fatality'] == 1]))

geoDf = gpd.GeoDataFrame(clusters, crs='EPSG:4326')
geoDf

,cluster,geometry,incident_count,inj/inc,inj
0,4,"POLYGON ((-83.06640 42.38398, -83.07272 42.386...",3536,0.011312,40
1,15,"POLYGON ((-83.14123 42.36964, -83.14409 42.369...",3923,0.011216,44
2,10,"POLYGON ((-83.04159 42.39389, -83.04161 42.393...",2363,0.013965,33
3,9,"POLYGON ((-83.10891 42.34184, -83.11273 42.342...",4357,0.011705,51
4,11,"POLYGON ((-83.19605 42.33463, -83.21077 42.336...",2949,0.006782,20
5,7,"POLYGON ((-83.17556 42.39579, -83.18863 42.395...",4623,0.007571,35
6,0,"POLYGON ((-82.96654 42.40492, -82.96654 42.409...",2987,0.009709,29
7,8,"POLYGON ((-83.14213 42.31369, -83.14228 42.313...",2306,0.010841,25
8,18,"POLYGON ((-83.03270 42.33103, -83.03527 42.331...",4367,0.007099,31
9,13,"POLYGON ((-83.23221 42.39263, -83.23349 42.392...",4373,0.009604,42


In [554]:
fig = go.Figure()


fig.add_trace(
    go.Choroplethmapbox(
        geojson=geoDf.__geo_interface__,
        locations=geoDf.cluster,
        featureidkey="properties.cluster",
        ids=geoDf.index,
        z=geoDf.incident_count,
        colorscale=[(0,"white"), (1,"red")],
        marker={'opacity':.6},
        name='Incident Cluster'
    )
)
fig.add_trace(
    go.Scattermapbox(
        lat=dfd_locations['Y'],
        lon=dfd_locations['X'],
        marker={'size':5, 'color':'black'},
        name='DFD Location'
    )
)
fig.update_layout(
    mapbox=dict(
        bearing=0,
        center = {"lat": 42.33, "lon": -83.05},
        pitch=0,
        zoom=10,
        style='light'
    ),
    mapbox_style="open-street-map"
)
#fig = px.scatter_mapbox(dfd_locations, lat="Y", lon="X", mapbox_style="open-street-map")
fig.show()

In [379]:
center = {"lat": 42.33, "lon": -83.05}
fig = px.choropleth_mapbox(geoDf,
                           geojson=geoDf.geometry,
                           locations=geoDf.index,
                           color="inj/inc",
                           
                           mapbox_style="open-street-map",
                           color_continuous_scale=[(0,"white"), (1,"red")],
                           center=center,
                           opacity=.6,
                           zoom=9)
fig.show()

In [304]:
dow_hour_counts = fire_inc.groupby(by=['DoW', 'hour'])['injury_or_fatality'].count().reset_index()
dow_hour_counts = dow_hour_counts.pivot(index='DoW', columns='hour', values='injury_or_fatality')
px.imshow(dow_hour_counts)

In [79]:
fire_inc.head()

,injury_or_fatality,call_datetime,x,y,hour,day,DoY,DoW,week,month,...,DoYx,DoYy,DoWx,DoWy,weekx,weeky,monthx,monthy,closest_stn,closest_light
6,0,2017-01-04 00:10:07+00:00,-83.088720,42.426064,0,4,4,2,1,1,...,0.068802,0.997630,0.974928,-0.222521,0.118273,0.992981,0.5,8.660254e-01,0.015584,0.015269
7,1,2017-01-10 09:23:12+00:00,-83.082236,42.418395,9,10,10,1,2,1,...,0.171293,0.985220,0.781831,0.623490,0.234886,0.972023,0.5,8.660254e-01,0.022253,0.016398
8,0,2017-01-14 08:03:22+00:00,-83.099447,42.443401,8,14,14,5,2,1,...,0.238673,0.971100,-0.974928,-0.222521,0.234886,0.972023,0.5,8.660254e-01,0.011979,0.020578
9,0,2017-03-02 16:32:33+00:00,-83.135443,42.380273,16,2,61,3,9,3,...,0.867456,0.497513,0.433884,-0.900969,0.875735,0.482792,1.0,6.123234e-17,0.013263,0.011673
10,0,2017-03-06 20:32:38+00:00,-83.045696,42.407905,20,6,65,0,10,3,...,0.899631,0.436651,0.000000,1.000000,0.926690,0.375828,1.0,6.123234e-17,0.016332,0.035715


In [303]:
month_day_counts = fire_inc.groupby(by=['month', 'day'])['injury_or_fatality'].count().reset_index()
month_day_counts = month_day_counts.pivot(index='month', columns='day', values='injury_or_fatality')
px.imshow(month_day_counts)

In [566]:
y_col = 'injury_or_fatality'
feature_cols = ['hourx', 'houry', 'dayx', 'dayy', 'DoYx', 'DoYy', 'DoWx', 'DoWy', 'weekx', 'weeky', 'monthx', 'monthy', 'closest_stn', 'closest_light']
for cluster in geoDf.cluster.unique():
    col = 'cluster_' + str(cluster)
    feature_cols.append(col)
    fire_inc[col] = fire_inc['cluster'].apply(lambda x: 1 if x == cluster else 0)
X = fire_inc.loc[:, feature_cols]
Y = fire_inc[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=RANDOM_STATE)

In [567]:
X_train.head()

,hourx,houry,dayx,dayy,DoYx,DoYy,DoWx,DoWy,weekx,weeky,...,cluster_6,cluster_3,cluster_12,cluster_16,cluster_14,cluster_1,cluster_19,cluster_2,cluster_17,cluster_5
66404,-1.000000,-1.836970e-16,-0.724793,0.688967,0.573772,-0.819015,-0.781831,0.623490,0.606800,-0.794854,...,0,0,0,0,0,0,0,0,0,0
58481,0.707107,7.071068e-01,-0.968077,-0.250653,0.790946,0.611886,0.974928,-0.222521,0.812487,0.582979,...,0,0,0,1,0,0,0,0,0,0
14450,1.000000,6.123234e-17,-0.998717,-0.050649,-0.991114,-0.133015,-0.974928,-0.222521,-0.978556,-0.205979,...,0,0,0,0,0,0,0,0,0,0
3416,0.866025,-5.000000e-01,0.937752,0.347305,0.594727,0.803928,0.000000,1.000000,0.652822,0.757511,...,0,0,1,0,0,0,0,0,0,0
19810,0.258819,9.659258e-01,0.937752,0.347305,0.425000,-0.905193,0.974928,-0.222521,0.403123,-0.915146,...,0,0,0,0,0,0,0,0,0,0


In [568]:
boost = xgb.XGBClassifier(use_label_encoder=False, objective="binary:logistic",
    n_estimators=1000,
    max_depth=3,
    learning_rate=0.1,
    scale_pos_weight=80,
    importance_type='total_gain',
    random_state=RANDOM_STATE)
boost.fit(X_train, y_train)

[15:44:28] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='total_gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=12, num_parallel_tree=1,
              random_state=123, reg_alpha=0, reg_lambda=1, scale_pos_weight=80,
              subsample=1, tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=None)

In [569]:
boost.save_model("boost_model.json")

In [536]:
boost.feature_importances_

array([0.11759345, 0.09091716, 0.03634458, 0.03339631, 0.02023849,
       0.00819576, 0.02029636, 0.00842122, 0.2651978 , 0.27551964,
       0.00688711, 0.00564372, 0.00303866, 0.00461531, 0.00570828,
       0.01004928, 0.00359507, 0.00735767, 0.01087043, 0.00663132,
       0.00411851, 0.01642257, 0.00564882, 0.00982011, 0.00492709,
       0.00353016, 0.        , 0.00744674, 0.00379057, 0.00377771],
      dtype=float32)

In [570]:
boost.get_booster().feature_names

['hourx',
 'houry',
 'dayx',
 'dayy',
 'DoYx',
 'DoYy',
 'DoWx',
 'DoWy',
 'weekx',
 'weeky',
 'monthx',
 'monthy',
 'closest_stn',
 'closest_light',
 'cluster_4',
 'cluster_15',
 'cluster_10',
 'cluster_9',
 'cluster_11',
 'cluster_7',
 'cluster_0',
 'cluster_8',
 'cluster_18',
 'cluster_13',
 'cluster_6',
 'cluster_3',
 'cluster_12',
 'cluster_16',
 'cluster_14',
 'cluster_1',
 'cluster_19',
 'cluster_2',
 'cluster_17',
 'cluster_5']

In [ ]:
cv = GridSearchCV(boost,
    param_grid={'max_depth':[2,3],
            'learning_rate':[ 0.1, 0.05],
            'scale_pos_weight':[80, 100]},
    scoring='roc_auc'
    )
cv.fit(X_train, y_train)

In [515]:
cv.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'scale_pos_weight': 80}

In [516]:
cv.best_score_

0.5498523637691932

In [575]:
boost_train_pred_proba = boost.predict_proba(X_train)[:,1]
get_model_metrics(y_train, boost_train_pred_proba, .35)

---Confusion Matrix---
[[TN  FP]
[FN  TP]]
[[40447  8088]
 [    0   512]]
---   ---
Accuracy: 0.835
AUC: 0.993
Precision: 0.06
Recall: 1.0


In [576]:
boost_test_pred_proba = boost.predict_proba(X_test)[:,1]
get_model_metrics(y_test, boost_test_pred_proba, .35)

---Confusion Matrix---
[[TN  FP]
[FN  TP]]
[[13349  2816]
 [  135    50]]
---   ---
Accuracy: 0.82
AUC: 0.566
Precision: 0.017
Recall: 0.27


In [283]:
log_reg = sm.Logit(y_train, X_train).fit()

Optimization terminated successfully.
         Current function value: 0.180977
         Iterations 11


In [284]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:     injury_or_fatality   No. Observations:                49047
Model:                          Logit   Df Residuals:                    49036
Method:                           MLE   Df Model:                           10
Date:                Thu, 19 Aug 2021   Pseudo R-squ.:                  -2.120
Time:                        17:12:24   Log-Likelihood:                -8876.4
converged:                       True   LL-Null:                       -2845.2
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
cluster          -0.1132      0.002    -57.435      0.000      -0.117      -0.109
DoYx              1.6615      0.642      2.589      0.010       0.404       2.919
DoYy            -30.5565      0.919    -33.251      0.000     -32.358     -28.755
DoWx              0.0236      0.029      0.811      0.418      -0.034       0.081
DoWy             -0.1718      0.029     -5.832      0.000      -0.230      -0.114
weekx            -2.3387      0.631     -3.707      0.000      -3.575      -1.102
weeky            30.0102      0.904     33.180      0.000      28.237      31.783
monthx            0.6772      0.181      3.739      0.000       0.322       1.032
monthy            0.8143      0.188      4.337      0.000       0.446       1.182
closest_stn    -187.7864      3.215    -58.411      0.000    -194.088    -181.485
closest_light   187.8000      3.215     58.412      0.000     181.498     194.101
=================================================================================

Possibly complete quasi-separation: A fraction 0.22 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [85]:
#training w/ bagging
RANDOM_STATE = 111
startingX = 13
y_col = 'injury_or_fatality'
X = fire_inc.iloc[:, startingX:]
Y = fire_inc[y_col]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=RANDOM_STATE)

batches = 1000
batchSize = int(.1 * len(X_train) )
recallThreshold = 0.5
score = 0
models = []
for batch in range(0,batches):
    X_batch_train, X_batch_test, Y_batch_train, Y_batch_test = train_test_split(
                                                                    X_train,
                                                                    Y_train,
                                                                    train_size=batchSize,
                                                                    test_size=batchSize)  #no random state so we get diff random batch
    C = 1
    linModel = LogisticRegression(random_state=RANDOM_STATE, C=C, penalty='l2', solver='liblinear', class_weight='balanced')
    linModel.fit(X_batch_train, Y_batch_train)
    
    Y_batch_pred = linModel.predict(X_batch_test)
    recall = metrics.recall_score(Y_batch_test, Y_batch_pred)

    #want to maximize recall to find all injuries
    if recall >= recallThreshold:
        models.append(linModel)

In [151]:
RANDOM_STATE = 111
startingX = 13

y_col = 'injury_or_fatality'
pos = fire_inc[fire_inc[ycol] == 1]
neg = fire_inc[fire_inc[ycol] == 0]
X_pos = pos.iloc[:,startingX:]
Y_pos = pos[y_col]
X_neg = neg.iloc[:,startingX:]
Y_neg = neg[y_col]
X_train_pos, X_test_pos, Y_train_pos, Y_test_pos = train_test_split(X_pos, Y_pos, random_state=RANDOM_STATE)
X_train_neg, X_test_neg, Y_train_neg, Y_test_neg = train_test_split(X_neg, Y_neg, random_state=RANDOM_STATE)

batchSize = len(X_train_pos)
batches = 10000
score = 0
models = []
for batch in range(0,batches):
    X_train_neg_batch, X_test_neg_batch, Y_train_neg_batch, Y_test_neg_batch = train_test_split(
                                                                                X_train_neg,
                                                                                Y_train_neg,
                                                                                train_size=batchSize,                                                                                                                                                  test_size=batchSize)
    X_batch = X_train_pos.append(X_train_neg_batch)
    Y_batch = Y_train_pos.append(Y_train_neg_batch)
    
    C = 1
    linModel = LogisticRegression(random_state=RANDOM_STATE, C=C, penalty='l2', solver='liblinear')
    linModel.fit(X_batch, Y_batch)
    
    X_test_batch = X_train_pos.append(X_test_neg_batch)
    Y_test_batch = Y_train_pos.append(Y_test_neg_batch)
    score = linModel.score(X_test_batch, Y_test_batch)

    if score >= .55:
        models.append(linModel)

X_test = X_test_pos.append(X_test_neg)
Y_test = Y_test_pos.append(Y_test_neg)
X_train = X_train_pos.append(X_train_neg)
Y_train = Y_train_pos.append(Y_train_neg)

In [153]:
#combine predictions from all models and avg probabilities
Y_pred = pd.DataFrame(Y_test)
cols = []
for idx, model in enumerate(models):
    pred = model.predict_proba(X_test)[:,1]
    Y_pred['model' + str(idx)] = pred
    cols.append('model' + str(idx))

Y_pred['pred'] = Y_pred[cols].mean(axis=1)
threshold = .5
true_pos = len(Y_pred[(Y_pred['injury_or_fatality'] == 1) & (Y_pred['pred'] >= threshold)])
true_neg = len(Y_pred[(Y_pred['injury_or_fatality'] == 0) & (Y_pred['pred'] < threshold)])
false_pos = len(Y_pred[(Y_pred['injury_or_fatality'] == 0) & (Y_pred['pred'] >= threshold)])
false_neg = len(Y_pred[(Y_pred['injury_or_fatality'] == 1) & (Y_pred['pred'] < threshold)])
conf_matrix = np.array([[true_neg, false_pos], [false_neg, true_pos]])
recall = true_pos / ( true_pos + false_neg )
print(conf_matrix)
print(recall)

[[9132 7043]
 [  69  106]]
0.6057142857142858


In [66]:
models[1].coef_

array([[ 0.6641071 , -0.35628719, -0.22662935,  0.10758354, -1.74409824,
         0.97414434,  0.0245776 , -0.2384062 ,  0.24592246,  0.44752147,
         1.43156447, -1.9779298 ,  0.45479227,  0.92237486, -2.92432499,
         1.39585009,  0.69807129,  0.3981502 ,  1.24850914, -0.8653115 ,
         0.68643348, -2.32200818, -0.10862503,  1.00827726,  0.4810092 ,
        -3.15185154,  0.30086039,  0.67322927, -0.20336876,  0.22082971,
         0.32348243,  0.27459983]])

In [67]:
X.columns

Index(['hourx', 'houry', 'dayx', 'dayy', 'DoYx', 'DoYy', 'DoWx', 'DoWy',
       'weekx', 'weeky', 'monthx', 'monthy', 'cluster_0', 'cluster_1',
       'cluster_2', 'cluster_3', 'cluster_4', 'cluster_5', 'cluster_6',
       'cluster_7', 'cluster_8', 'cluster_9', 'cluster_10', 'cluster_11',
       'cluster_12', 'cluster_13', 'cluster_14', 'cluster_15', 'cluster_16',
       'cluster_17', 'cluster_18', 'cluster_19'],
      dtype='object')

In [499]:
##single logisitic regression w/o bagging
C = 10
linModel = LogisticRegression(random_state=RANDOM_STATE, C=C, penalty='l2', solver='liblinear', class_weight='balanced')
linModel.fit(X_train, Y_train)
Y_pred = linModel.predict(X_test)

## [[TN  FP]
#   [FN  TP]]
get_model_metrics(y_train, linModel.predict_proba(X_train)[:,1])
get_model_metrics(y_test, linModel.predict_proba(X_test)[:,1], .55)

---Confusion Matrix---
[[TN  FP]
[FN  TP]]
[[25726 22809]
 [  239   273]]
---   ---
Accuracy: 0.53
AUC: 0.544
Precision: 0.012
Recall: 0.533
---Confusion Matrix---
[[TN  FP]
[FN  TP]]
[[14970  1195]
 [  165    20]]
---   ---
Accuracy: 0.917
AUC: 0.5
Precision: 0.016
Recall: 0.108


In [275]:
def get_model_metrics(y_true, y_pred_proba, threshold=0.5):
    y_pred = (y_pred_proba > threshold).astype(int)
    print("---Confusion Matrix---")
    print("""[[TN  FP]\n[FN  TP]]""")
    print(metrics.confusion_matrix(y_true, y_pred))
    print("---   ---")
    print(f'Accuracy: {round(metrics.accuracy_score(y_true, y_pred),3)}')
    print(f'AUC: {round(metrics.roc_auc_score(y_true, y_pred_proba),3)}')
    print(f'Precision: {round(metrics.precision_score(y_true, y_pred),3)}')
    print(f'Recall: {round(metrics.recall_score(y_true, y_pred),3)}') 

In [113]:
(np.array([.4, .6]) > .5).astype(int)

array([0, 1])